In [1]:
import tensorflow as tf
from tensorflow import keras
import gymnasium
from gymnasium.wrappers import AtariPreprocessing
from gymnasium.wrappers import FrameStackObservation
import ale_py
import matplotlib.pyplot as plt
import numpy as np

2024-11-04 16:23:28.247481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-04 16:23:28.921293: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nick/miniconda3/envs/tf_env/lib/python3.9/site-packages/nvidia/cudnn/lib:/home/nick/miniconda3/envs/tf_env/lib/
2024-11-04 16:23:28.921391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nick

In [2]:
# actions = [NOOP, FIRE, RIGHT, LEFT]
# step returns [state, reward, terminated, truncated, info]

In [3]:
env = gymnasium.make("BreakoutNoFrameskip-v4", render_mode="rgb_array")

A.L.E: Arcade Learning Environment (version 0.10.1+unknown)
[Powered by Stella]


In [4]:
obs, info = env.reset()
obs.shape

(210, 160, 3)

In [5]:
class AtariPreprocessingFire(AtariPreprocessing):
    def reset(self, **kwargs):
        obs, reset_info = super().reset(**kwargs)
        super().step(1)
        return obs, reset_info
    def step(self, action):
        self.lives_before_action = self.ale.lives()
        obs, rewards, terminated, truncated, info = super().step(action)
        done = terminated or truncated
        if not done and self.ale.lives() < self.lives_before_action:
            super().step(1)
        return obs, rewards, terminated, truncated, info

In [6]:
env = AtariPreprocessingFire(env)
env = FrameStackObservation(env, stack_size=4)

In [7]:
obs, info = env.reset()

In [8]:
for _ in range(4):
    time_step = env.step(3)

In [9]:
state,reward,terminated,truncated,_ = time_step

In [10]:
def plot_observation(obs):
    obs = obs.astype(np.float32)
    img = obs[:3]
    current_frame_delta = np.maximum(obs[3] - obs[:3].mean(axis=0), 0.)
    img[0] += current_frame_delta
    img[2] += current_frame_delta
    img = np.clip(img / 150, 0, 1)
    img = np.transpose(img, (1,2,0))
    plt.imshow(img)

In [11]:
# (channels, height, width)

In [12]:
q_net = keras.models.Sequential([
    keras.layers.Input(shape=env.observation_space.shape),
    keras.layers.Lambda(lambda obs: tf.cast(obs, np.float32) / 255.),
    keras.layers.Conv2D(32, (8,8), strides=4, activation="relu", data_format="channels_first"),
    keras.layers.Conv2D(64, (4,4), strides=2, activation="relu", data_format="channels_first"),
    keras.layers.Conv2D(64, (3,3), strides=1, activation="relu", data_format="channels_first"),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(4)
])

In [13]:
update_period = 4
optimizer = keras.optimizers.RMSprop(learning_rate=2.5e-4, rho=0.95, momentum=0.0, epsilon=1e-5, centered=True)
epsilon_fn = keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=1.0,
                                          decay_steps=250000 // update_period,
                                          end_learning_rate=0.01)

In [20]:
def epsilon_greedy_policy(obs, action_space, epsilon):
    if np.random.rand() < epsilon:
        return action_space.sample()
    else:
        q_values = q_net.predict(obs)
        return np.argmax(q_values)    

In [ ]:
from collections import deque
replay_buffer = deque(maxlen=100000)
def sample_experiences():
    

In [15]:
obs, _ = env.reset()

In [16]:
q_net(obs[np.newaxis])

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[-0.03705218,  0.00442155, -0.01860621, -0.03340802]],
      dtype=float32)>

In [17]:
state, rewards, truncated, terminated, _ = env.step(3)

In [18]:
q_net(state[np.newaxis])

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[-0.03859906,  0.00465813, -0.01861539, -0.03358131]],
      dtype=float32)>

In [19]:
env.observation_space.shape

(4, 84, 84)

In [21]:
env.observation_space.

Box(0, 255, (4, 84, 84), uint8)